In [ ]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

from scipy.stats import chi2_contingency

from dython import nominal

import warnings

warnings.filterwarnings('ignore')

In [ ]:
# Loading the dataset

df = pd.read_csv('./Dataset/vehicles.csv')

df.head(3)

In [ ]:
df = pd.read_excel('filename.xlsx')

### Lets have a quick look at the dataset

In [ ]:
df.info()

In [ ]:
df.describe().T

We can realize some odds based on the data description above. We might need to double check if the vehicle has 3965000 mileage. Moreover, the min price is quite low (at 100), the max price is extremly high (at 1399888). The min passengers is 0, and the max passenger is 17. These points don't make sense.

#### We will drop some columns which we will not use

In [ ]:
# Checking if id column has only unique values

df['id'].nunique() == df.shape[0]

In [ ]:
# Checking if dataset has duplicated rows

df.loc[df.duplicated('id')]

In [ ]:
unnecessary_cols = ['id', 'first_date_seen', 'last_date_seen', 'vin', 'trim', 'description',
                    'carfax_url', 'province', 'engine', 'longitude', 'latitude']

df.drop(unnecessary_cols, axis = 1, inplace = True)

df.head(3)

#### Missing Values

In [ ]:
# Nan is the list of features, have null values

Nan = df.isnull().sum()[df.isnull().sum() > 0]

Nan.sort_values().plot(kind = 'barh', figsize = (8, 8), color = "#3F5D7D");

for y, x in enumerate(Nan.sort_values()):
    
    plt.annotate(str(round(x/df.shape[0]*100,2))+'%', xy=(x, y), size=10)
    
plt.xlabel('The Number of Missing Values')

plt.ylabel('Feature Names')

In [ ]:
# Dropping Null rows in some columns

df.dropna(subset = ['color', 'price', 'seller_name', 'mileage'], inplace = True)

df.reset_index(drop = True, inplace = True)

In [ ]:
df.head(3)

#### Create a function to check if two categorical columns are correlated

In [ ]:
def check_categorical_correlated(col1, col2):
    
    tmp_crosstab = pd.crosstab(index = df[col1], columns = df[col2])
    
    tmp_crosstab_result = chi2_contingency(tmp_crosstab)
    
    if round(tmp_crosstab_result[1], 2) < 0.05:
        
        result = col1 + ' and ' + col2 + ' are correlated'
        
        # Set missing values to the most occurrence of its corresponding correlated column
        
        for i in range(len(df.year)):

            if (pd.isnull(df[col1][i])) | (df[col1][i] == 0):

                if df[col2][i] in tmp_crosstab.columns.tolist():

                    df.loc[i, col1] = tmp_crosstab[df[col2][i]].idxmax()    

        # Double-check if there are null values in body_type column

        df_tmp = df.loc[df['body_type'].isnull()]

        if df_tmp.shape[0] < 2500:

            # Dropping those rows

            df.dropna(subset = [col1], inplace = True)

            df.reset_index(drop = True, inplace = True)                    
                    
    else:
        
        result = col1 + ' and ' + col2 + ' are not correlated'
        
    return result

#### Relationship between model and body_type

In [ ]:
check_categorical_correlated('body_type', 'model')

In [ ]:
# Double-check again

df['body_type'].unique()

#### Relationship between passengers and body_type

In [ ]:
check_categorical_correlated('passengers', 'body_type')

In [ ]:
# Double-check again

df['passengers'].unique()

#### Let's see what type of car has 17 passenger seats

In [ ]:
df.loc[df['passengers'] == 17]

It does not make sense that Crosstrek has 17 seats. Thereby, we will drop this one

In [ ]:
df.drop(df.loc[df['passengers'] == 17].index, inplace = True)

df.reset_index(drop = True, inplace = True)

# Double-check passengers column again

df['passengers'].unique()

#### We will do some transformation to make data more readable

In [ ]:
# Removing any whitespaces in the beginning and ending of a string
# Titling strings
# Upper-case names

for col in ['model', 'color', 'fuel_type', 'seller_name']:
    
    df[col] = df[col].apply(lambda row: row.rstrip().lstrip().title() if not pd.isnull(row) else row)
    
for col in ['make', 'body_type', 'city']:
    
    df[col] = df[col].apply(lambda row: row.rstrip().lstrip().upper() if not pd.isnull(row) else row)

#### Normalizing data in some columns for future use

In [ ]:
df['drivetrain'] = df['drivetrain'].apply(lambda x: x.replace('4x4', 'AWD').replace('4WD', 'AWD').replace('2WD', 'FWD').replace('4X4', 'AWD') if not pd.isnull(x) else x)

# Double-check drivetrain column again

df['drivetrain'].unique()

In [ ]:
for i in range(len(df.year)):
    
    if not pd.isnull(df['transmission'][i]):
    
        if 'Automatic' in df['transmission'][i]:

            df.loc[i, 'transmission'] = 'Automatic'

        elif 'Manual' in df['transmission'][i]:

            df.loc[i, 'transmission'] = 'Manual'

        elif 'CVT' in df['transmission'][i]:

            df.loc[i, 'transmission'] = 'CVT'   
        
# Double-check transmission column again

df['transmission'].unique()

In [ ]:
# fuel_type column

df['fuel_type'].unique()

We can tell "Flexible", "Other", and "Other/Don't Know" are noise data

In [ ]:
df.loc[df['fuel_type'].isin(["Flexible", "Other", "Other/Don’T Know"])].shape

We can remove those rows because the number of rows is not high

In [ ]:
df.drop(df.loc[df['fuel_type'].isin(["Flexible", "Other", "Other/Don’T Know"])].index, inplace=True)

df.reset_index(drop = True, inplace = True)

# Double-check fuel_type column again

df['fuel_type'].unique()

#### Relationship between model and drivetrain

In [ ]:
check_categorical_correlated('drivetrain', 'model')

In [ ]:
# Double-check again

df['drivetrain'].unique()

#### Relationship between model and drivetrain

In [ ]:
check_categorical_correlated('transmission', 'model')

In [ ]:
# Double-check again

df['transmission'].unique()

#### Relationship between model and fuel_type

In [ ]:
check_categorical_correlated('fuel_type', 'model')

In [ ]:
# Double-check again

df['fuel_type'] = df['fuel_type'].apply(lambda x: x.replace('Gasoline', 'Gas').replace('Gas Fuel', 'Gas').replace('Gas - Hybrid', 'Gas Hybrid'))

df['fuel_type'].unique()

#### Convert boolean values in is_private column to a column of integers 1 or 0

In [ ]:
df['is_private'] = df['is_private'].astype(int)

In [ ]:
# Double-check again

df['is_private'].unique()

#### There are no null values in all columns

In [ ]:
df.info()

#### Valid Original Equipment Manufacturers

In [ ]:
df['make'].unique()

#### Year has no significance on its own. Thus, we need to extract how old a car is and see how its price might be affected

In [ ]:
# how many years old the car is.

df['car_age'] = 2022 - df['year']

# Drop the year column

df.drop(['year'], axis = 1, inplace = True)

#### Let's go back to the odds which found in previous data description

We will create a function to remove all outliers

In [ ]:
def outlier_points(col):
    
    data = sorted(df[col].unique().tolist())
    
    # Calculate first(q1) and third quartile(q3)
    
    q1, q3 = np.percentile(data, [25,75])
    
    # Find interquartile range (q3-q1)
    
    iqr = q3 - q1
    
    lower_bound = q1 - (1.5 * q1)
    
    upper_bound = q3 + (1.5 * q3)
    
    return lower_bound, upper_bound

In [ ]:
x, y = outlier_points('mileage')

print(x)

print(y)

In [ ]:
# Drop all outliers for mileage column

df.drop(df.loc[(df['mileage'] < x) | (df['mileage'] > y)].index, inplace = True)

df.reset_index(drop = True, inplace = True)

In [ ]:
a, b = outlier_points('price')

print(a)

print(b)

In [ ]:
# Drop all outliers for price column

df.drop(df.loc[(df['price'] < a) | (df['price'] > b)].index, inplace = True)

df.reset_index(drop = True, inplace = True)

In [ ]:
c, d = outlier_points('car_age')

print(c)

print(d)

In [ ]:
# Drop all outliers for car_age column

df.drop(df.loc[(df['car_age'] < c) | (df['car_age'] > d)].index, inplace = True)

df.reset_index(drop = True, inplace = True)

#### Have a quick look at data description again

In [ ]:
df.describe().T

In [ ]:
plt.figure(figsize = (10,8))

sns.distplot(df['price'], color = "#3F5D7D")

plt.ylabel('Frequency')

plt.title("Distribution of the variable 'price'")

plt.axvline(df['price'].mean(), linestyle = '--', color = 'r', label = 'mean')

plt.axvline(df['price'].median(), linestyle = '--', color = 'b', label = 'median')

plt.legend()

In [ ]:
print("Skewness: %f" % df['price'].skew())

The distribution of prices shows a high positive skewness to the left (skew > 1). The issue might be in regards to homoscedasticity and assumption violations. Log(price) would be a good solution to have a more visualization of the distribution of the price. Moreover, log(price) has no negative side effects on the prediction.

In [ ]:
df['price'] = np.log(df['price'])

#### We will save down the cleaned file for future uses.

In [ ]:
df.to_csv('./Dataset/vehicles-cleaned.csv', index = False)